In [ ]:
import numpy as np

INSPIRATION FROM THE FOLLOWING BLOG 

https://medium.com/aimonks/traveling-salesman-problem-tsp-using-genetic-algorithm-fea640713758

In [487]:
from random import randrange, random

In [439]:
# set up country list

cities = [
    "Paris", "Berlin", "London", "Madrid", "Rome", "Amsterdam", "Lisbon", "Prague", "Vienna", "Stockholm",
    "Athens", "Budapest", "Warsaw", "Oslo", "Copenhagen", "Dublin", "Brussels", "Zurich", "Munich", "Barcelona",
    "Helsinki", "Bratislava", "Luxembourg", "Reykjavik", "Geneva", "Milan", "Dubrovnik", "Edinburgh", "Krakow",
    "Bucharest", "Riga", "Sofia", "Hamburg", "Istanbul", "Kiev", "Ljubljana", "Tallinn", "Vilnius", "Valletta",
    "Nicosia", "Belgrade", "Cologne", "Rotterdam", "The Hague", "Antwerp", "Marseille", "Minsk", "Belfast", "Cork",
    "Glasgow",
]

len(cities)

50

In [440]:
def determine_coords(city_list, max_distance):
    country_coords = {}
    for city in city_list:
        while len(country_coords) < len(city_list):
            coords = (randrange(0, max_distance), randrange(0, max_distance))
            if coords not in country_coords.values():
                country_coords[city] = coords
                break

    return country_coords

coords = determine_coords(cities, 20)
print(len(coords))

50


In [441]:
# out of above countries, select N countries to construct initial population

def initial_countries(n_sol, n_countries, country_list):
    selected_cities = []  # list of n countries to permute
    while len(selected_cities) < n_sol:
        sublist_cities = []  # list of n countries to permute
        while len(sublist_cities) < n_countries:
            rand = randrange(0, len(country_list))
            selected_city = country_list[rand]
            if selected_city not in sublist_cities:
                sublist_cities.append(selected_city)
        if len(set(sublist_cities)) == n_countries:  # Ensure unique cities
            selected_cities.append(sublist_cities)

    return selected_cities

cities_selection = initial_countries(1000, 10, cities)

In [442]:
def construct_pop_coords(solution_arr, coord_map):
    result_dict_list = []

    for subarray in solution_arr:
        subarray_dict = {city: coord_map.get(city, None) for city in subarray}
        result_dict_list.append(subarray_dict)

    return result_dict_list

solutions_coords = construct_pop_coords(cities_selection, coords)

In [443]:
# terribly slow, might want to refactor this whole block... (3 loops just really bad)

def calc_total_dist(sol_coords):
    total_dist_per_sol = []
    for sol_coord in sol_coords:
        first_coords = list(sol_coord.values())[0]  # Get coordinates of the first city
        solution_total_dist = []
        for coords in sol_coord.values():
            distance = ((first_coords[0] - coords[0])**2 + (first_coords[1] - coords[1])**2)**0.5
            solution_total_dist.append(distance)
        total_dist_per_sol.append(solution_total_dist)

    return total_dist_per_sol

ts = calc_total_dist(solutions_coords)

In [444]:
def calc_weights(dist_list):
    total_dist_list = []
    for sub_array in dist_list:
        total_dist = sum(sub_array)
        total_dist_list.append(total_dist)

    return total_dist_list

test = calc_weights(ts)

In [565]:
def assign_weights(distance_sum_list):
    score_list = []
    for distance_sum in distance_sum_list:
        score_list.append(1 / (1 + distance_sum))
        
    return score_list

weights = assign_weights(test)

len(weights)

print(max(weights))

0.024964353228455527


In [512]:
def probability_wheel(solutions, weights_list):
    total_score = sum(weights_list)
    probabilities = [x / total_score for x in weights_list] # gets proportion of each solution based on score (adds to 1)
    selection = []
    for i in range(len(solutions)):
        rand = random() # number between 0 and 1
        prob_sum = 0 # cumulative probability
        for idx, prob in enumerate(probabilities):
            prob_sum += prob # adds up to total
            if prob_sum > rand: # cutoff is exceeded
                selection.append(solutions[idx])
                break

    return selection

selected_solutions = probability_wheel(solutions_coords, weights)

In [551]:
def crossover(solutions, cross_rate): # TO DO : IMPLEMENT CROSSOVER RATE
    parents = [(solutions[i], solutions[i + 1]) for i in range(0, len(solutions), 2)]
    cross_solutions = [] # store the newly crossovered solutions
    for parent1, parent2 in parents:
        sub_cross_solutions_c1 = {}
        sub_cross_solutions_c2 = {}

        p1_city = [city for city in parent1.keys()]
        p2_city = [city for city in parent2.keys()]
        p1_coords = [coord for coord in parent1.values()]
        p2_coords = [coord for coord in parent2.values()]

        rand = randrange(1, len(solutions[0]))

        child_1_cities_cross = p1_city[:rand]
        child_2_cities_cross = p2_city[rand:]
        child_1_coord_cross = p1_coords[:rand]
        child_2_coord_cross = p2_coords[rand:]

        # this was in fact found in the blog mentioned in the sources above, i was just unaware the "+=" could be used on lists
        child_1_cities_cross += [city for city in child_2_cities_cross if city not in child_1_cities_cross]
        child_2_cities_cross += [city for city in child_1_cities_cross if city not in child_2_cities_cross]
        child_1_coord_cross += [coord for coord in child_2_coord_cross if coord not in child_1_coord_cross]
        child_2_coord_cross += [coord for coord in child_1_coord_cross if coord not in child_2_coord_cross]

        # this block of code reverts the crossover if there were any duplicates (code above was problematic)
        if child_1_cities_cross != len(p1_city):
            child_1_cities_cross = list(parent1.keys()) # if unable to get to length 10, kill crossover
            child_1_coord_cross = list(parent1.values())
        if child_2_cities_cross != len(p2_city):
            child_2_cities_cross = list(parent2.keys())
            child_2_coord_cross = list(parent2.values())

        for idx, city in enumerate(child_1_cities_cross):
            sub_cross_solutions_c1[city] = child_1_coord_cross[idx]
        for idx, city in enumerate(child_2_cities_cross):
            sub_cross_solutions_c2[city] = child_2_coord_cross[idx]

        cross_solutions.append(sub_cross_solutions_c1)
        cross_solutions.append(sub_cross_solutions_c2)

    return cross_solutions

cross_sample = crossover(selected_solutions, 0.5)

In [559]:
def mutate(city_coord_map, solutions, mutate_rate):
    mutated_solutions = []
    city_names = [city for city in city_coord_map.keys()]
    city_coords = [coords for coords in city_coord_map.values()]
    for solution in solutions:
        cities = [city for city in solution.keys()]
        coords = [coord for coord in solution.values()]
        for idx, current_city in enumerate(cities):
            rand = random()
            rand_idx = randrange(0, len(city_coord_map))
            new_city = city_names[rand_idx]
            new_coord = city_coords[rand_idx]
            if rand < mutate_rate and new_city not in cities:
                cities[idx] = new_city
                coords[idx] = new_coord

        mutated_solutions.append(dict(zip(cities, coords))) # maybe refactor this thing at some point if we want to

    return mutated_solutions

mutated_results = mutate(coords, cross_sample, 0.10)

print(len(mutated_results))

1000


In [568]:
import numpy as np

def genetic_algorithm(cities, max_distance, n_sol, n_countries, n_iterations, cross_rate, mutate_rate):
    # Step 1: Generate initial population
    coords = determine_coords(cities, max_distance)
    cities_selection = initial_countries(n_sol, n_countries, cities)
    solutions_coords = construct_pop_coords(cities_selection, coords)

    optimal_solution = None
    optimal_score = float('-inf')

    for iteration in range(n_iterations):
        # Step 2: Calculate scores and probabilities
        total_distances = calc_total_dist(solutions_coords)
        weights = assign_weights(calc_weights(total_distances))
        selected_solutions = probability_wheel(solutions_coords, weights)

        # Step 3: Crossover
        crossover_results = crossover(selected_solutions, cross_rate)

        # Step 4: Mutation
        mutated_results = mutate(coords, crossover_results, mutate_rate)

        # Step 5: Report scores and total distance
        total_distances_after = calc_total_dist(mutated_results)
        weights_after = assign_weights(calc_weights(total_distances_after))

        # Find the best score and solution in the current iteration
        max_score_index = np.argmax(weights_after)
        max_score = weights_after[max_score_index]
        max_solution = mutated_results[max_score_index]
        total_distance = sum(total_distances_after[max_score_index])

        print(f"Iteration {iteration + 1}: Max Score = {max_score}, Total Distance = {total_distance}")

        # Update optimal solution if a better one is found
        if max_score > optimal_score:
            optimal_score = max_score
            optimal_solution = max_solution

        # Update the current solutions for the next iteration
        solutions_coords = mutated_results

    # Print the optimal solution after all iterations
    print(f"Optimal Solution: {optimal_solution}, Optimal Score: {optimal_score}")

    return mutated_results


# Example usage
max_distance = 20
n_sol = 1000
n_countries = 10
n_iterations = 10000
cross_rate = 0.5
mutate_rate = 0.1

genetic_algorithm(cities, max_distance, n_sol, n_countries, n_iterations, cross_rate, mutate_rate)

Iteration 1: Max Score = 0.018795688433414934, Total Distance = 52.203691024277795
Iteration 2: Max Score = 0.01931523418048146, Total Distance = 50.77260553281439
Iteration 3: Max Score = 0.020576444840848647, Total Distance = 47.599260354965985
Iteration 4: Max Score = 0.019819610792212384, Total Distance = 49.45507757361839
Iteration 5: Max Score = 0.020129653721584654, Total Distance = 48.67795342290059
Iteration 6: Max Score = 0.020129653721584654, Total Distance = 48.67795342290059
Iteration 7: Max Score = 0.020129653721584654, Total Distance = 48.67795342290059
Iteration 8: Max Score = 0.020469790089649872, Total Distance = 47.85247946463454
Iteration 9: Max Score = 0.02200251235690786, Total Distance = 44.44935522720169
Iteration 10: Max Score = 0.02200251235690786, Total Distance = 44.44935522720169
Iteration 11: Max Score = 0.02200251235690786, Total Distance = 44.44935522720169
Iteration 12: Max Score = 0.022659778005396567, Total Distance = 43.13105899633455
Iteration 13: M

[{'Riga': (5, 9),
  'The Hague': (2, 1),
  'Munich': (4, 9),
  'Valletta': (7, 12),
  'Athens': (14, 7),
  'London': (2, 11),
  'Amsterdam': (5, 11),
  'Hamburg': (18, 16),
  'Berlin': (14, 13),
  'Cork': (1, 10)},
 {'Riga': (5, 9),
  'Paris': (7, 9),
  'Dubrovnik': (6, 7),
  'Milan': (1, 2),
  'The Hague': (2, 1),
  'Amsterdam': (5, 11),
  'Belgrade': (2, 8),
  'Warsaw': (10, 5),
  'Barcelona': (5, 16),
  'Valletta': (7, 12)},
 {'Riga': (5, 9),
  'Ljubljana': (2, 10),
  'Antwerp': (14, 4),
  'Reykjavik': (13, 15),
  'Helsinki': (1, 12),
  'Zurich': (12, 9),
  'Edinburgh': (7, 3),
  'Lisbon': (8, 17),
  'Berlin': (14, 13),
  'Valletta': (7, 12)},
 {'Riga': (5, 9),
  'Vilnius': (17, 5),
  'Vienna': (19, 3),
  'Prague': (9, 3),
  'Ljubljana': (2, 10),
  'Helsinki': (1, 12),
  'The Hague': (2, 1),
  'Geneva': (7, 19),
  'Amsterdam': (5, 11),
  'Lisbon': (8, 17)},
 {'Riga': (5, 9),
  'Dubrovnik': (6, 7),
  'Cologne': (12, 3),
  'Vienna': (19, 3),
  'Kiev': (14, 16),
  'Zurich': (12, 9),
  